In [8]:
# imports
import arcpy
from tqdm.notebook import tqdm
from arcpy.sa import *

# ArcGIS setup
arcpy.env.overwriteOutput = True
arcpy.env.extent = "MAXOF"

# set an existing raster to copy attributes from
existing_raster = r"E:/fires/fires.gdb/US_fire_count"

# set the snap raster setting
arcpy.env.snapRaster = existing_raster

# set workspace path
workspace = r"E:/fires/fires.gdb/"
arcpy.env.workspace = workspace

# root folder
root_path = r"E:/fires/"

# contains MTBS data 1984-2018 as rasters
data_path = r"E:/fires/mtbs/"

# check out the spatial analyst tools
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

# convert to binary

we only care about the most severe fires (rated as 4 in the rasters). We thus mask out all values that are not zero using [raster algebra](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/raster-calculator.htm) and a [conditional statement](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/con-.htm). Unfortunately, this process is extremely inefficient even on high-end hardware. Classic ArcPy.

In [2]:
# get a list of files to process
raw_mtbs_list = glob.glob(data_path + "*.tif")

# loop through all the files to process
for mtbs_raster in tqdm(raw_mtbs_list):

    # set as a raster
    mtbs_raster = arcpy.Raster(mtbs_raster)
    
    # convert to a binary mask of only "4" (high intensity fires)
    binary_raster = Con(mtbs_raster, 0, 1, "VALUE < 4")
    
    # get a cleaner savename
    save_name = arcpy.Describe(mtbs_raster).basename + "_binary"
    
    # save the binary raster
    binary_raster.save(save_name)

# combine into one map

This produces a map where each pixel value represents the number of times that cell has been severely burned. We use a combination of [cell statistics](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/cell-statistics.htm) to sum the 34 years together, and then [project the raster](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/project-raster.htm) to make sure everything is in the same projection system for later.

In [4]:
# get a list of rasters
binary_raster_list = arcpy.ListRasters("*binary")

# sum the rasters together
sum_binary_raster = CellStatistics(binary_raster_list, "SUM", "DATA")
print("Rasters combined. Projecting raster...")

# set the save name
save_name = workspace + "binary_sum"

# save and project the results to match the rest of the data
arcpy.ProjectRaster_management(sum_binary_raster, save_name, existing_raster, "NEAREST")
print("Projection finished!")

Rasters combined. Projecting raster...
Projection finished!


# smooth and downsample the raster
There are simply too many pixels in the binary sum raster to convert directly to points here. We smooth the raster to with different [kernels](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/focal-statistics.htm) the size of the larger cells (24:1 ratio). We can then more accurately [downsample](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/resample.htm) that smoothed raster to the resolution of the other data.

In [ ]:
# define a function to save a few lines of code
def apply_smoothing(kernel_function, save_name):

    # apply a 24x24  kernel
    smoothed_raster = FocalStatistics(workspace + "binary_sum", NbrRectangle(24, 24, "CELL"), kernel_function, "DATA")

    # save the smoothed raster
    smoothed_raster.save(workspace + save_name)
    
    # get the cell size from the existing raster
    cell_size = "0.0083333333333 0.0083333333333"
    
    # downsample the smoothed raster
    arcpy.Resample_management(smoothed_raster, workspace + save_name + "_resample", cell_size, "NEAREST")
    
apply_smoothing("MEAN", "binary_sum_mean")
apply_smoothing("MEDIAN", "binary_sum_median")
apply_smoothing("SUM", "binary_sum_sum")

In [ ]:
# check in the spatial analyst tools
arcpy.CheckInExtension("Spatial")